In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

import sys
sys.path.append('/home/rustam/hyperspecter_segmentation/makitorch')

In [2]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import IncrementalPCA
from hsi_dataset_api import HsiDataset

from makitorch.dataloaders.HsiDataloader import HsiDataloader

from tqdm import tqdm

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train_indices = np.load('data/kfold2_indx_train.npy')

In [4]:
BATCH_SIZE = 32
dataset_hsi_loader = HsiDataloader(
    '/raid/rustam/hyperspectral_dataset/new_cropped_hsi_data', 
    indices=train_indices,
    shuffle_data=True,
)

data_loader = torch.utils.data.DataLoader(dataset_hsi_loader, batch_size=BATCH_SIZE)

In [5]:
ipca = IncrementalPCA(n_components=17, whiten=True, batch_size=BATCH_SIZE)

iterator = tqdm(data_loader)

for img_batch, _ in iterator:
    # (N, C, H, W) --> (N, H, W, C)
    img_batch = img_batch.permute(0, 2, 3, 1)
    # (N, H, W, C) --> (N * H * W, C)
    img_batch = img_batch.reshape(-1, img_batch.shape[-1]).numpy()
    ipca.partial_fit(img_batch)
iterator.close()

6it [17:51, 178.54s/it]


In [6]:
ipca.components_

array([[ 0.00574835,  0.0060122 ,  0.00626566, ...,  0.01313406,
         0.01347327,  0.01384627],
       [ 0.01512712,  0.01680658,  0.01785282, ...,  0.00096721,
         0.00091768,  0.00091608],
       [-0.02285927, -0.02398215, -0.02454252, ..., -0.0338058 ,
        -0.03310942, -0.03268486],
       ...,
       [-0.04952149, -0.04730101, -0.04611028, ..., -0.03551168,
        -0.02929392, -0.02532326],
       [-0.04572145, -0.04558128, -0.04204976, ..., -0.05407019,
        -0.05234802, -0.04205934],
       [-0.02361013, -0.01940358, -0.01499578, ..., -0.02122218,
        -0.02086222, -0.02073657]])

In [7]:
ipca.components_.shape

(17, 237)

In [8]:
ipca.explained_variance_

array([1.73538368e+05, 3.15140188e+04, 2.98223137e+03, 1.25418636e+03,
       7.57441946e+02, 4.29743152e+02, 2.34572154e+02, 2.18572276e+02,
       1.37720699e+02, 1.26521094e+02, 6.42988639e+01, 4.98609971e+01,
       3.99558403e+01, 2.33675735e+01, 1.99581304e+01, 1.60651840e+01,
       1.28171544e+01])

In [9]:
ipca.mean_ 

array([  4.51935929,   4.80161579,   5.02678952,   5.32928533,
         5.65460011,   5.93797931,   6.26567018,   6.60816844,
         6.81085579,   7.01186972,   7.29501373,   7.59144566,
         7.848941  ,   8.0543867 ,   8.2246508 ,   8.35293983,
         8.52313606,   8.82176103,   9.19071309,   9.57801058,
         9.9591506 ,  10.15472266,  10.1207848 ,   9.99886318,
        10.2749168 ,  10.84063615,  11.40019426,  11.89140605,
        12.2861446 ,  12.49925665,  12.64028515,  12.91633562,
        13.3786103 ,  13.94441087,  14.55503588,  15.11802414,
        15.5289067 ,  16.04815624,  16.9486935 ,  18.06949935,
        19.16083271,  20.26702795,  21.56078774,  22.9024115 ,
        24.01132122,  24.94967176,  25.61914482,  25.94823693,
        26.31206379,  26.66613935,  27.01736732,  27.58954616,
        28.3882942 ,  29.12754494,  29.59059297,  29.76230531,
        29.76982792,  29.49958911,  28.9552366 ,  28.36575587,
        27.90232657,  27.79452291,  27.94273744,  28.08

In [12]:
np.save('data/kfold2_PcaExplainedVariance_.npy', ipca.explained_variance_)
np.save('data/kfold2_PcaMean.npy', ipca.mean_)
np.save('data/kfold2_PcaComponents.npy', ipca.components_)

In [13]:
ipca.explained_variance_

array([1.73538368e+05, 3.15140188e+04, 2.98223137e+03, 1.25418636e+03,
       7.57441946e+02, 4.29743152e+02, 2.34572154e+02, 2.18572276e+02,
       1.37720699e+02, 1.26521094e+02, 6.42988639e+01, 4.98609971e+01,
       3.99558403e+01, 2.33675735e+01, 1.99581304e+01, 1.60651840e+01,
       1.28171544e+01])